In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

print(torch.cuda.get_device_name())
print(torch.__version__, torch.cuda.is_available())
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

NVIDIA RTX A2000 8GB Laptop GPU
2.0.0+cu118 True
cuda:0


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
x_data = pd.read_csv('../data/env1/x_extract.csv', delimiter=',',encoding="utf-8-sig", header=None).values.tolist()
y_data = pd.read_csv('../data/env1/y_extract.csv', delimiter=',',encoding="utf-8-sig", header=None).values.tolist()
theta_data = pd.read_csv('../data/env1/theta_extract.csv', delimiter=',',encoding="utf-8-sig", header=None).values.tolist()

In [58]:
#make input data and ouptput data
input_data_1 = []
input_data_2 = []
output_data = []

for i in range(np.shape(x_data)[0]):
    input_data_2_list = [x_data[i][0], x_data[i][-1], y_data[i][0], y_data[i][-1], theta_data[i][0], theta_data[i][-1]]
    for j in range(1, np.shape(x_data)[1]):
        x = x_data[i][0:j]
        y = y_data[i][0:j]
        input_data_1_list = []
        for k in range(len(x)):
            input_data_1_list.append([x[k], y[k]])
        input_data_1.append(input_data_1_list)
        input_data_2.append(input_data_2_list)
        output_data.append([x_data[i][j], y_data[i][j]])


In [47]:
input_data_1[2]

[[-1.764922937, 2.257040621],
 [-0.784877232, 2.552430241],
 [0.352215063, 2.895154319]]

In [46]:
input_data_2[2]

[-1.764922937, 29.18854837, 2.257040621, 0.716586999, 0.292744299, 0.839522787]

In [59]:
class MyDataset(Dataset):
    def __init__(self, input_data_1, input_data_2, output_data):
        super(MyDataset, self).__init__()
        self.input_data_1 = input_data_1
        self.input_data_2 = input_data_2
        self.output_data = output_data

    def __getitem__(self, index):
        input_1 = torch.tensor(self.input_data_1[index], dtype=torch.float32)
        input_2 = torch.tensor(self.input_data_2[index], dtype=torch.float32)
        output = torch.tensor(self.output_data[index], dtype=torch.float32)
        return input_1, input_2, output

    def __len__(self):
        return len(self.input_data_1)

In [60]:
train_valid_data = MyDataset(input_data_1, input_data_2, output_data)

val_size = 2000
train_data, val_data = torch.utils.data.random_split(train_valid_data, [len(train_valid_data)-val_size, val_size])

In [61]:
np.shape(train_data[0][2])

torch.Size([2])

In [62]:
train_data[0]

(tensor([[ 0.0859,  0.3388],
         [ 0.0859,  0.3388],
         [ 0.6437, -1.5818],
         [ 2.2623, -2.5050],
         [ 3.7339, -2.2085],
         [ 4.7066, -1.5221],
         [ 5.5161, -0.7597],
         [ 6.2683,  0.0424],
         [ 7.0621,  0.9023],
         [ 8.1799,  1.9895]]),
 tensor([ 0.0859, 28.0988,  0.3388,  2.1004, -1.2882, -1.1793]),
 tensor([9.0733, 2.5118]))

In [65]:
batch_size = 1

dataloader_train = torch.utils.data.DataLoader(
    train_data,
    batch_size=batch_size,
    shuffle=True
)

dataloader_valid = torch.utils.data.DataLoader(
    val_data,
    batch_size=batch_size,
    shuffle=True
)

In [53]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

In [54]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, output_seq_length, num_layers):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.output_seq_length = output_seq_length
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=self.num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        #入力のsequence長さを出力のsequence長さに合わせるため拡張する
        new_size = (x.size(0), self.output_seq_length, input_size)
        x = x.expand(*new_size)
        
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        output, _ = self.lstm(x, (h0, c0))
        output = self.fc(output)

        return output

In [ ]:
class LSTM_MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, output_seq_length, num_layers):
        super(LSTM_MLP, self).__init__()
        self.hidden_size = hidden_size
        self.output_seq_length = output_seq_length
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=self.num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x1, x2):
        #入力のsequence長さを出力のsequence長さに合わせるため拡張する
        new_size = (x.size(0), self.output_seq_length, input_size)
        x = x.expand(*new_size)
        
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        output, _ = self.lstm(x, (h0, c0))
        output = self.fc(output)

        return output

In [67]:
import torch
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence

# 入力シーケンスのリストを作成
sequences = [torch.tensor([1, 2, 3]), torch.tensor([4, 5]), torch.tensor([6, 7, 8, 9])]

# パディング
padded_sequences = pad_sequence(sequences, batch_first=True)

# パディングのマスキング
padding_mask = (padded_sequences != 0).float()

# パック・パディング・シーケンス
packed_sequences = pack_padded_sequence(padded_sequences, lengths=[3, 2, 4], batch_first=True, enforce_sorted=False)

# バッチ化
batched_sequences = packed_sequences.batch_sizes

# 出力を確認
print("Padded Sequences:\n", padded_sequences)
print("Padding Mask:\n", padding_mask)
print("Packed Sequences:\n", packed_sequences)
print("Batched Sequences:\n", batched_sequences)

Padded Sequences:
 tensor([[1, 2, 3, 0],
        [4, 5, 0, 0],
        [6, 7, 8, 9]])
Padding Mask:
 tensor([[1., 1., 1., 0.],
        [1., 1., 0., 0.],
        [1., 1., 1., 1.]])
Packed Sequences:
 PackedSequence(data=tensor([6, 1, 4, 7, 2, 5, 8, 3, 9]), batch_sizes=tensor([3, 3, 2, 1]), sorted_indices=tensor([2, 0, 1]), unsorted_indices=tensor([1, 2, 0]))
Batched Sequences:
 tensor([3, 3, 2, 1])


torch.Tensor